In [1]:
import os, shlex
from subprocess import check_output
repo_rootdir = check_output(shlex.split("git rev-parse --show-toplevel")).strip().decode('ascii')
os.chdir(repo_rootdir)

import os
import sys
import torch

from config_parser import read_config
from helpers import *

import mesh_operations
device = get_device()
from IPython.display import display

import ipywidgets as widgets

import pyvista as pv
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy

ModuleNotFoundError: No module named 'psbody'

In [2]:
output_dir = "output"
experiments = [x for x in sorted(os.listdir(output_dir)) if os.path.exists(os.path.join(output_dir, x, ".finished"))]
# experiments = [x for x in sorted(os.listdir(output_dir))]# if os.path.exists(os.path.join(output_dir, x, ".finished"))]
w = widgets.Dropdown(
    value='2020-08-27_04-58-31',
    options=experiments,
    description='Experiment:',
    disabled=False,
)

display(w)

Dropdown(description='Experiment:', options=('2020-08-27_04-58-31', '2020-08-27_04-59-39', '2020-08-27_04-59-4…

In [3]:
import ExperimentClass
# Load experiment
experiment = ExperimentClass.ComaExperiment(os.path.join(output_dir, w.value))
experiment.load_model()

2020-09-16 15:53:01,858 [ERROR] Unsupported point attribute type: point_data for file: ./template/template.vtk


In [4]:
# Load meshes
prealigned_meshes = experiment.load_prealigned_meshes()
dataloader = get_loader(prealigned_meshes.vertices, prealigned_meshes.ids, batch_size=8, num_workers=4, shuffle=False)

# map subject ID with position
id_dict = {k:v for v,k in enumerate(prealigned_meshes.ids)}

In [5]:
def get_mesh_for_id(dataloader, id, id_dict):
    position = id_dict[id]
    return dataloader.dataset[position][0].unsqueeze(0)

def reconstruct(exper, mesh):

    model = exper.model
    with torch.no_grad():
        if model.is_variational:
            mu, log_var = model.encoder(x=mesh)
            z = mu
        else:
            z = model.encoder(x=mesh)
        mesh_r = model.decoder(z)
    return mesh_r

In [6]:
wcb=widgets.Combobox(
    placeholder="Choose a subject",
    options=sorted(prealigned_meshes.ids),
    value="1000336"
)

In [12]:
def plot_mesh(mesh):
  kargs = {"point_size": 5, "render_points_as_spheres": True}
  plotter = pv.Plotter(notebook=True)
  plotter.add_mesh(mesh, **kargs)
  plotter.show(interactive=True)
  plotter.enable()

def f(id, reconstructed=True, deviation=True):
  try:
    mesh = get_mesh_for_id(dataloader, id, id_dict)
    _mesh = reconstruct(experiment, mesh) if reconstructed else mesh
    _mesh = _mesh.detach().numpy() if deviation else prealigned_meshes.mean + prealigned_meshes.std * _mesh.detach().numpy()
    plot_mesh(_mesh.squeeze(0))
  except:
    pass
    
interact(f, 
  id=wcb,
  reconstructed=True,
  deviation=True
)

interactive(children=(Combobox(value='4855542', description='id', options=('1000336', '1000380', '1000407', '1…

<function __main__.f(id, reconstructed=True, deviation=True)>

In [9]:
def reconstruct_from_z(experiment, z):
    model = experiment.model
    with torch.no_grad():
        mesh_r = model.decoder(z)
    return mesh_r    

In [10]:
def g(z0, z1, z2, z3, z4, z5, z6, z7, deviation=True):
    
    z = torch.Tensor([z0, z1, z2, z3, z4, z5, z6, z7])    
    mesh = reconstruct_from_z(experiment, z.unsqueeze(0)) # get_mesh_for_id(dataloader, id, id_dict)
    _mesh = mesh.detach().numpy() if deviation else prealigned_meshes.mean + prealigned_meshes.std * mesh.detach().numpy()
    plot_mesh(_mesh.squeeze(0))

z_sliders = {"z"+str(i):widgets.FloatSlider(value=0, min=-3, max=3, step=1) for i in range(experiment.config['z'])}

interact(g, 
  **z_sliders,
  deviation=True
)

interactive(children=(FloatSlider(value=0.0, description='z0', max=3.0, min=-3.0, step=1.0), FloatSlider(value…

<function __main__.g(z0, z1, z2, z3, z4, z5, z6, z7, deviation=True)>

In [ ]:
z = torch.Tensor([0]*8)
mesh = reconstruct_from_z(experiment, z.unsqueeze(0))
grid = pv.StructuredGrid(mesh[0,:,0], mesh[0,:,1], mesh[0,:,2])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
plotter = pv.Plotter()
plotter.add_mesh(grid, smooth_shading=True)

print('Orient the view, then press "q" to close window and produce movie')

# setup camera and close
plotter.show(auto_close=False)